In [81]:
import os
import locale
import math

import pandas as pd
import numpy as np

from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from antecedentes import create_antecedentes_doc

import pypdf
from pathlib import Path

from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate

from docx2pdf import convert

In [2]:
import subprocess

def generate_pdf(doc_path, path):

    subprocess.call(['soffice',
                  '--headless',
                 '--convert-to',
                 'pdf',
                 '--outdir',
                 path,
                 doc_path])
    return doc_path

In [3]:
def get_file_path(row):
    return Path('/media/informacoes_pdf/') / row['setor'].strip() / row['arquivo']


In [13]:
def get_pdf_text(row):
    arquivo = get_file_path(row)
    print(f'File {arquivo} to pypdf text')
    try:
        pdf = pypdf.PdfReader(arquivo)
        text = []
        for page in pdf.pages:
            text.append(page.extract_text())
        return ' '.join(text)
    except FileNotFoundError:
        print(f'File not found: {arquivo}')
        return ''

In [14]:
antecedentes_sql = ''' 

SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.interessado,
       pm.Descricao as marcador      
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) like '%antecedente%'
'''
processos_antecedentes = pd.read_sql(antecedentes_sql, conn)

/tmp/ipykernel_19154/146702693.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_antecedentes = pd.read_sql(antecedentes_sql, conn)


In [15]:
processos_antecedentes

,numero_processo,ano_processo,assunto,interessado,marcador
0,000761,2022,APURAÇÃO DE RESPONSABILIDADE EM ATENDIMENTO AO...,PREFEITURA MUNICIPAL DE CANGUARETAMA ...,Antecedentes
1,000913,2023,COMUNICAÇÃO DE IRREGULARIDADE ...,SIGILOSO ...,Antecedentes
2,001749,2019,"CONTAS DO CHEFE DO PODER EXECUTIVO MUNICIPAL, ...",PREFEITURA MUNICIPAL DE VIÇOSA ...,Antecedentes
3,003277,2020,AUMENTO REMUNERATÓRIO DURANTE A PANDEMIA - REM...,CÂMARA MUNICIPAL DE ANTÔNIO MARTINS ...,Antecedentes
4,003980,2022,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE CEARÁ-MIRIM ...,Antecedentes
5,004072,2021,MONITORAMENTO ...,AGÊNCIA DE FOMENTO DO RN S/A ...,Antecedentes
6,005181,2020,AUDITORIA DE CONFORMIDADE-EXERCÍCIO 2018 ...,PREFEITURA MUNICIPAL DE MOSSORÓ ...,Antecedentes
7,006621,2015,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.UPANEMA ...,Antecedentes


In [16]:
sql_info_antecedentes_gaana = ''' 
WITH RankedData AS (
    SELECT 
    	ai.setor,
        concat(pro.numero_processo, '/', pro.ano_processo) as processo,
        pro.assunto,
        pro.interessado,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        ai.datapublicacao, 
        ppe.SequencialProcessoEvento, 
        ai.numero_processo, 
        ai.ano_processo,
        ROW_NUMBER() OVER (
            PARTITION BY CONCAT(ai.numero_processo, '/', ai.ano_processo)
            ORDER BY ppe.SequencialProcessoEvento DESC
        ) AS rn
    FROM processo.dbo.Ata_Informacao ai 
    LEFT JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON ai.idInformacao = ppe.IdInformacao
    LEFT JOIN processo.dbo.Processos pro
    	ON pro.numero_processo = ai.numero_processo AND pro.ano_processo = ai.ano_processo
    WHERE CONCAT(ai.numero_processo, '/', ai.ano_processo) IN (
        SELECT CONCAT(pro.numero_processo, '/', pro.ano_processo)
        FROM processo.dbo.Processos pro
        INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
            ON pmp.IdProcesso = pro.IdProcesso
        INNER JOIN processo.dbo.Pro_Marcador pm 
            ON pmp.IdMarcador = pm.IdMarcador
        LEFT JOIN processo.dbo.Exe_Debito ed 
            ON ed.IdProcessoExecucao = pro.IdProcesso
        LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
            ON edp.IDDebito = ed.IdDebito
        LEFT JOIN processo.dbo.GenPessoa gp 
            ON gp.IdPessoa = edp.IDPessoa
        WHERE pro.setor_atual = 'CCD'
        AND lower(pm.Descricao) LIKE '%antecedente%'
    )
    AND ai.setor LIKE '%GAANA%'
)
SELECT * 
FROM RankedData 
WHERE rn = 1
ORDER BY numero_processo, ano_processo;


'''
df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)

/tmp/ipykernel_19154/2358130519.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)


In [99]:
def get_transito(nome):
    with open("consultas/processos_transito_cpf.sql") as f:
        query = f.read()
    with get_connection() as conn:
        q = query.format(nome=nome)
        #print(q)
        df = pd.read_sql(q, conn)
        df['valor_original'] = df['valor_original'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df['valor_atualizado'] = df['valor_atualizado'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df.fillna('', inplace=True)
        return df.to_dict(orient='records')
    
def get_processos_transito_by_cpf(pessoas_cpfs):
    transitos = []
    for pessoa in pessoas_cpfs:
        if pessoa['nome']:
            transitos.append({'nome': pessoa['nome'], 'cpf': pessoa['cpf'], 'transitos': get_transito(pessoa['nome'])})
    return transitos   

def get_genpessoa(pessoa_strc):
    cpfs = []
    for pessoa in pessoa_strc.pessoas:
        nome_pessoa = pessoa.nome_pessoa
        sql = f'''
        SELECT gp.Documento
            FROM Processo.dbo.GenPessoa gp 
            WHERE lower(gp.nome) = '{nome_pessoa.lower()}'
        '''
        ret = pd.read_sql(sql, get_connection())
        if ret.shape[0] == 0 or ret.shape[0] > 1:
            cpfs.append({'nome': nome_pessoa, 'cpf': None})
        else:
            cpfs.append({'nome': nome_pessoa, 'cpf': ret.iloc[0]['Documento']})
    return cpfs

def is_vazio(valores):
    return all([not bool(v['transitos']) for v in valores])
    

In [21]:
df_info_gaana['texto_pdf'] = df_info_gaana.apply(get_pdf_text, axis=1)

File /media/informacoes_pdf/GAANA/GAANA_000761_2022_0028.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_000913_2023_0027.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_001749_2019_0042.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003277_2020_0124.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003980_2022_0029.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004072_2021_0039.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_005181_2020_0084.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_006621_2015_0037.pdf to pypdf text


In [22]:
llm = AzureChatOpenAI(model_name="gpt-4o")

In [23]:
from typing import List

from pydantic import BaseModel, Field

class PessoaAntecedentes(BaseModel):
  """Pessoa cujo antecedente deve ser buscado"""
  nome_pessoa: str = Field(description="Nome da pessoa para buscar antecedentes")

class PessoasAntecedentes(BaseModel):
  """Lista de pessoas cujos antecedentes devem ser buscados"""
  pessoas: List[PessoaAntecedentes] = Field(description="Lista de pessoas para buscar antecedentes")


In [24]:
def get_pessoas_llm(texto_despacho):

  prompt = PromptTemplate.from_template("""
    Você é um agente que identifica listas de pessoas em requisições de antecedentes. 
      Você recebeu uma requisição para buscar antecedentes de pessoas em um processo. Remova qualquer vocativo do nome da pessoa, tal como Sr., Sra., Dr., Dra., etc.
      O texto da requisição é o seguinte:
      "{input}"
      Encontre os antecedentes das pessoas listadas na requisição.

    Sua resposta:
    """)

  structured_llm = llm.with_structured_output(schema=PessoasAntecedentes)
  chain = prompt | structured_llm
  return chain.invoke(texto_despacho)

In [25]:
df_info_gaana['pessoas'] = df_info_gaana['texto_pdf'].apply(get_pessoas_llm)

In [49]:
df_info_gaana['cpf'] = df_info_gaana['pessoas'].apply(get_genpessoa)

/tmp/ipykernel_19154/1252809892.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_19154/1252809892.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_19154/1252809892.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())


In [82]:
df_info_gaana['valores'] = df_info_gaana['cpf'].apply(get_processos_transito_by_cpf)

/tmp/ipykernel_19154/1905053526.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_19154/1905053526.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_19154/1905053526.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_19154/1905053526.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not te

In [ ]:
df_info_gaana['vazio'] = df_info_gaana['valores'].apply(is_vazio)

0    False
1     True
2    False
3    False
4     True
5     True
6    False
7    False
Name: valores, dtype: bool

In [98]:
df_info_gaana['valores'].iloc[5]

[{'nome': 'Edivane Vilar de Souza', 'cpf': '45583005491', 'transitos': []}]

In [83]:
df_info_gaana.valores[2]

[{'nome': 'Ana Paula de Oliveira Gomes',
  'cpf': '45639027304',
  'transitos': []},
 {'nome': 'Antônio Gomes de Amorim',
  'cpf': '18249604415',
  'transitos': [{'processo_origem': '009308/2004',
    'processo_execucao': '',
    'tipo_debito': 'Ressarcimento ',
    'valor_original': '61.950,00',
    'valor_atualizado': '-',
    'transito_julgado': '13/03/2013',
    'situacao_divida': 'Cancelada por Decisão Judicial',
    'status_protesto': '',
    'status_pge': ''},
   {'processo_origem': '009308/2004',
    'processo_execucao': '',
    'tipo_debito': 'Multa Percentual',
    'valor_original': '18.585,00',
    'valor_atualizado': '-',
    'transito_julgado': '13/03/2013',
    'situacao_divida': 'Cancelada por Decisão Judicial',
    'status_protesto': '',
    'status_pge': ''},
   {'processo_origem': '009308/2004',
    'processo_execucao': '',
    'tipo_debito': 'Multa',
    'valor_original': '2.200,00',
    'valor_atualizado': '2.222,00',
    'transito_julgado': '13/03/2013',
    'situa

In [58]:
df_info_gaana.cpf.iloc[0]

[{'nome': 'Maria de Fátima Borges Marinho', 'cpf': '12627690434'}]

In [ ]:
df_info_gaana['processo'] = df_info_gaana['numero_processo'] + '/' + df_info_gaana['ano_processo']


In [ ]:
df_info_gaana.iloc[0]

setor                                                                   GAANA
arquivo                                            GAANA_000761_2022_0028.pdf
datapublicacao                                     2025-02-07 08:51:56.317000
SequencialProcessoEvento                                                   51
numero_processo                                                        000761
ano_processo                                                             2022
rn                                                                          1
texto_pdf                                \n \n1 \n \n \nProcesso nº: 761/2...
pessoas                     pessoas=[PessoaAntecedentes(nome_pessoa='Maria...
cpf                                                               12627690434
Name: 0, dtype: object

In [ ]:
for i,row in df_info_gaana.iterrows():
    row['valores'] = get_processos_transito_by_cpf(row['cpf'])

/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query.format(cpf=cpf), conn)
/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query.format(cpf=cpf), conn)
/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query.format(cpf=cpf), conn)
/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI

/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query.format(cpf=cpf), conn)
/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query.format(cpf=cpf), conn)
/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query.format(cpf=cpf), conn)
/tmp/ipykernel_30906/941986394.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI